# Pre-processing of LAS tiles

Pre-processing consists of two parts:
1. Clipping of AHN point clouds to match target point clouds;
2. Ground and building surface extraction.

In [1]:
# Helper script to allow importing from parent folder.
import set_path

# Import modules.
import numpy as np
import time

import src.preprocessing.ahn_preprocessing as ahn_preprocessing
import src.utils.ahn_utils as ahn_utils
import src.utils.las_utils as las_utils
import src.utils.plot_utils as plot_utils

In [32]:
# Set-up.

# The point cloud tile for which a corresponding AHN tile should be created.
tile_code = '2426_9774'
filename = '../datasets/pointcloud/labelled' + tile_code + '.laz'

# The folder where AHN data will be saved.
ahn_data_folder = '../datasets/ahn/'

## 1a. Clip a single file

In [3]:
# Download the corresponding AHN cloud from 
# https://www.arcgis.com/apps/Embed/index.html?appid=a3dfa5a818174aa787392e461c80f781
!wget https://download.pdok.nl/rws/ahn3/v1_0/laz/C_25DN2.LAZ -P ../datasets/

--2022-01-13 15:19:50--  https://download.pdok.nl/rws/ahn3/v1_0/laz/C_25DN2.LAZ
Resolving download.pdok.nl (download.pdok.nl)... 51.138.23.105
Connecting to download.pdok.nl (download.pdok.nl)|51.138.23.105|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2944451484 (2,7G) [application/octet-stream]
Saving to: ‘../datasets/C_25DN2.LAZ’

C_25DN2.LAZ           6%[>                   ] 171,43M   526KB/s    eta 1h 42m ^C


In [35]:
# Load the AHN point cloud.
# WARNING: this may require 16+ GB of RAM, depending on the size of the cloud.
ahn_cloud_file = '../datasets/AHN/25EZ1_22.LAZ'
ahn_cloud = las_utils.read_las(ahn_cloud_file)

In [36]:
# Clip the corresponding AHN point cloud tile.
start = time.time()
ahn_preprocessing.clip_ahn_las_tile(ahn_cloud, filename, out_folder=ahn_data_folder, buffer=1)
end = time.time()
print(f'Tile clipped in {end-start:.2f} seconds.')

Tile clipped in 0.47 seconds.


## 1b. Clip all files in a folder

In [18]:
las_folder = '../datasets/pointcloud/'

# Set resume=True to skip existing files in the out_folder.
ahn_preprocessing.clip_ahn_las_folder(ahn_cloud, las_folder, out_folder=ahn_data_folder, buffer=1, resume=True, prefix='labelled_')

100%|██████████| 2/2 [08:23<00:00, 251.68s/file]


## 2a. Surface extraction for a single file

In [59]:
tile_code = '2441_9726'

In [60]:
# Load the clipped tile that we have just created.
ahn_las_file = ahn_data_folder + '/ahn_' + tile_code + '.laz'

start = time.time()
# This will create the AHN ground and building surfaces, with a 0.1m resolution, and save them as an .npz file.
_file = ahn_preprocessing.process_ahn_las_tile(ahn_las_file, out_folder=ahn_data_folder)
end = time.time()
print(f'Processed LAS tile in {end-start:.2f}s.')

Processed LAS tile in 3.68s.


In [61]:
# Load the result.
ahn_reader = ahn_utils.NPZReader(ahn_data_folder)

In [62]:
# Optional: plot the resulting surfaces.
%matplotlib widget
plot_utils.plot_ahn_sidebyside(tile_code, ahn_reader)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

This will look like:

![Surface Plot](https://github.com/Amsterdam-AI-Team/Urban_PointCloud_Processing/raw/main/media/examples/surface_plot.png)

## 2b. Surface extraction for a folder of ahn cloud tiles

We have a batch script to do this, it allows using multiple CPU threads to speed things up.

```sh
$ cd ../scripts
$ python ahn_batch_processor.py --in_folder ../datasets/ahn/ --out_folder ../datasets/ahn/ --resume --workers 4
```

In [2]:
ahn_data_folder = '../datasets/ahn/'
start = time.time()

tiles = las_utils.get_tilecodes_from_folder(las_folder)
for tile_code in tiles:
    ahn_las_file = ahn_data_folder + '/ahn_' + tile_code + '.laz'

    # This will create the AHN ground and building surfaces, with a 0.1m resolution, and save them as an .npz file.
    _file = ahn_preprocessing.process_ahn_las_tile(ahn_las_file, out_folder=ahn_data_folder)
end = time.time()
print(f'Processed LAS tile in {end-start:.2f}s.')

Processed LAS tile in 13.71s.
